In [4]:
#| default_exp classes.DomoInstanceConfig

In [5]:
#| exporti
import httpx
import datetime as dt
import asyncio
from fastcore.basics import patch_to


from dataclasses import dataclass, field
from typing import List

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoError as de
import domolibrary.routes.instance_config as instance_config_routes
import domolibrary.routes.grant as grant_routes
import domolibrary.classes.DomoGrant as dmdg
import domolibrary.classes.DomoRole as dmr
import domolibrary.routes.role as role_routes

# import Library.utils.convert as cd
# from .DomoAuth import DomoAuth
# from .DomoApplication import DomoApplication
# import Library.DomoClasses.DomoPublish as dmpb


# import domolibrary.utils.convert as cd
# import domolibrary.utils.DictDot as util_dd
# import domolibrary.client.DomoError as de


In [6]:
# | export
@dataclass
class DomoInstanceConfig:
    """utility class that absorbs many of the domo instance configuration methods"""
    
    auth: dmda.DomoAuth
    allowlist : list[str] = field(default_factory = list)

In [7]:
# | export
@patch_to(DomoInstanceConfig)
async def get_allowlist(self: DomoInstanceConfig, 
                        auth: dmda.DomoFullAuth = None, # get_allowlist requires full authentication
                        session: httpx.AsyncClient = None, 
                        return_raw : bool = False,
                        debug_api: bool = False) -> list[str]:
    """retrieves the allowlist for an instance"""

    auth = auth or self.auth

    res =  await instance_config_routes.get_allowlist(auth=auth,  debug_api=debug_api, session = session)

    if return_raw:
        return res

    if not res.is_success:
        return None

    allowlist = res.response.get('addresses')

    self.allowlist = allowlist

    return allowlist


#### sample implementation of get_allowlist

In [8]:
import os

## this route requires full auth
# auth = dmda.DomoTokenAuth(domo_instance = 'domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

auth = dmda.DomoFullAuth(domo_instance= 'domo-dojo', domo_username= 'jae@onyxreporting.com', domo_password= os.environ['DOJO_PASSWORD'])

dmicnfg = DomoInstanceConfig(auth = auth)

await dmicnfg.get_allowlist(auth=auth)


['0.0.0.0/0', '20.125.101.170']

## ALTER the allowlist

- The updating the allowlist will automatically include the IP Address that submits the set_allowlist request.  This prevents users from accidentally locking themselves out of domo.
- the set_allowlist API always does a SET command, by default there is no UPDATE or INSERT option.  This class-based implementation uses a combination of get_allowlist, MERGE and set_allowlist to perform an UPSERT equivalent.


In [9]:
# |export
@patch_to(DomoInstanceConfig)
async def set_allowlist(self : DomoInstanceConfig,
                        ip_address_ls: list[str],
                        debug_api: bool = False,
                        auth: dmda.DomoFullAuth = None,
                        session: httpx.AsyncClient = None
                        ):
                        
    auth = auth or self.auth

    await instance_config_routes.set_allowlist(auth=auth,
                                               ip_address_ls=ip_address_ls,
                                               debug_api=debug_api, session=session)

    return await self.get_allowlist(auth=auth, debug_api=debug_api, session=session)


@patch_to(DomoInstanceConfig, )
async def upsert_allowlist(self : DomoInstanceConfig,
                           ip_address_ls: list[str],
                           debug_api: bool = False,
                           session: httpx.AsyncClient = None,
                           auth: dmda.DomoAuth = None,
                           ):

    exist_ip_address_ls = await self.get_allowlist(auth=auth, debug_api=debug_api, session=session)
    ip_address_ls += exist_ip_address_ls

    return await self.set_allowlist(auth=auth,
                                   ip_address_ls=list(set(ip_address_ls)),
                                   debug_api=debug_api, session=session)


#### sample implementation of set and upsert allowlist

In [10]:
import os

# this route requires full auth
# auth = dmda.DomoTokenAuth(domo_instance = 'domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

auth = dmda.DomoFullAuth(domo_instance='domo-dojo',
                         domo_username='jae@onyxreporting.com', domo_password=os.environ['DOJO_PASSWORD'])

dmicnfg = DomoInstanceConfig(auth = auth)

await dmicnfg.set_allowlist(auth=auth, ip_address_ls=['0.0.0.0/0'])


['0.0.0.0/0', '20.125.123.201']

In [11]:
import os

# this route requires full auth
# auth = dmda.DomoTokenAuth(domo_instance = 'domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

auth = dmda.DomoFullAuth(domo_instance='domo-dojo',
                         domo_username='jae@onyxreporting.com', domo_password=os.environ['DOJO_PASSWORD'])

dmicnfg = DomoInstanceConfig(auth = auth)

await dmicnfg.upsert_allowlist(auth=auth, ip_address_ls=['192.168.0.1'])


['0.0.0.0/0', '192.168.0.1', '20.125.123.201']

## Domo Grants

In [12]:
# | export
@patch_to(DomoInstanceConfig)
async def get_grants(self: DomoInstanceConfig,
                     auth: dmda.DomoAuth = None,
                     debug_prn:bool = False,
                     debug_api: bool = False,
                     session: httpx.AsyncClient = None,
                     return_raw: bool = False):

    auth = auth or self.auth

    res = await grant_routes.get_grants(auth=auth,
                                        debug_api=debug_api,
                                        session=session)

    if debug_prn:
        print(
            f"ℹ️ - get_instance_grants: {len(res.response)} grants returned from {auth.domo_instance}")
    


    if return_raw:
        return res

    if res.status == 200:
        json_list = res.response
        return [dmdg.DomoGrant._from_json(obj) for obj in json_list]


In [22]:
import os
import pandas as pd 

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-dojo",
)

dmicnfg = DomoInstanceConfig(auth = auth)

res = await dmicnfg.get_grants(debug_prn= True)

pd.DataFrame(res[0:5])

ℹ️ - get_instance_grants: 84 grants returned from domo-dojo


DomoGrant(id='fileshare.create', display_group='DATA', title='Create Fileshare Directories', depends_on_ls=[], description='Create, edit and delete Fileshare directories that you can access.', role_membership_ls=['1', '2', '3', '275763436', '810756122'])

## Domo Roles

In [ ]:
# | export
@patch_to(DomoInstanceConfig)
async def get_roles(self, auth: dmda.DomoAuth = None,
                    debug_api: bool = False,
                    return_raw: bool = False,
                    session: httpx.AsyncClient = None):

    auth = auth or self.auth

    res = await role_routes.get_roles(auth=auth,
                                      debug_api=debug_api, session = session)
    
    if return_raw:
        return res

    if res.status == 200:
        json_list = res.response
        return [dmr.DomoRole._from_json(obj = obj, auth = auth
                                   ) for obj in json_list]


#### sample implementation of get_roles

In [ ]:

import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(domo_instance = 'domo-dojo', domo_access_token= os.environ['DOMO_DOJO_ACCESS_TOKEN'])

dmdic = DomoInstanceConfig(auth = token_auth)

res = await dmdic.get_roles( return_raw = False)

pd.DataFrame(res)

,auth,id,name,description,is_system_role,is_default_role,grant_ls,membership_ls
0,"{'domo_instance': 'domo-dojo', 'domo_access_to...",1,Admin,Full access to everything,True,None,[],[]
1,"{'domo_instance': 'domo-dojo', 'domo_access_to...",2,Privileged,Full access except for editing users and setti...,True,None,[],[]
2,"{'domo_instance': 'domo-dojo', 'domo_access_to...",3,Editor,"Can edit Cards, Pages, DataSets, and Dataflows",True,None,[],[]
3,"{'domo_instance': 'domo-dojo', 'domo_access_to...",4,Participant,"Read only access on Cards, Pages DataSets and ...",True,None,[],[]
4,"{'domo_instance': 'domo-dojo', 'domo_access_to...",5,Social,Access limited to social features,True,None,[],[]
5,"{'domo_instance': 'domo-dojo', 'domo_access_to...",104295428,super_admin_v3,upsert via DomoLibrary,0,None,[],[]
6,"{'domo_instance': 'domo-dojo', 'domo_access_to...",275763436,Super_Admin,Full access to everything,0,None,[],[]
7,"{'domo_instance': 'domo-dojo', 'domo_access_to...",810756122,super_admin,updated via domolibrary on 2023-03-20,0,None,[],[]
8,"{'domo_instance': 'domo-dojo', 'domo_access_to...",1563101750,dl_department_admin,deployed via domo_library script - updated 202...,0,None,[],[]
9,"{'domo_instance': 'domo-dojo', 'domo_access_to...",1662852841,dl_test,deployed via domo_library script - updated 202...,0,None,[],[]


## Authorized Domains

In [ ]:
# | export

@patch_to(DomoInstanceConfig)
async def get_authorized_domains(self: DomoInstanceConfig,
                                 auth: dmda.DomoAuth = None, 
                                 debug_api: bool = False,
                                 session: httpx.AsyncClient = None,
                                 return_raw :bool = False
                                 ):
    auth = auth or self.auth
                                 
    res = await instance_config_routes.get_authorized_domains(auth=auth,
                                                              debug_api=debug_api,
                                                              session=session,
                                                              return_raw = return_raw
                                                              )

    if return_raw:
        return res

    return res.response


#### sample implementation of get_authorized_domains

In [ ]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(domo_instance = 'domo-dojo', domo_access_token= os.environ['DOMO_DOJO_ACCESS_TOKEN'])

dmdic = DomoInstanceConfig(auth = token_auth)

await dmdic.get_authorized_domains( return_raw = False)


ResponseGetData(status=200, response=[], is_success=True)

In [ ]:
# #| export
# @dataclass
# class DomoConnector:
#     id: str
#     label: str
#     title: str
#     sub_title: str
#     description: str
#     create_date: dt.datetime
#     last_modified: dt.datetime
#     publisher_name: str
#     writeback_enabled: bool
#     tags: list[str] = field(default_factory=list)
#     capabilities: list[str] = field(default_factory=list)

#     @classmethod
#     def _from_str(cls, obj):
#         dd = DictDot(obj)

#         return cls(
#             id=dd.databaseId,
#             label=dd.label,
#             title=dd.title,
#             sub_title=dd.subTitle,
#             description=dd.description,
#             create_date=cd.convert_epoch_millisecond_to_datetime(
#                 dd.createDate),
#             last_modified=cd.convert_epoch_millisecond_to_datetime(
#                 dd.lastModified),
#             publisher_name=dd.publisherName,
#             writeback_enabled=dd.writebackEnabled,
#             tags=dd.tags,
#             capabilities=dd.capabilities
#         )

In [ ]:

    # @classmethod
    # async def get_connectors(cls, auth: DomoAuth,
    #                          session: httpx.AsyncClient = None,
    #                          debug: bool = False,
    #                          limit=100,
    #                          ):

    #     is_close_session = False
    #     if not session:
    #         is_close_session = True
    #         session = httpx.AsyncClient()

    #     def arr_fn(res):
    #         # pprint(res.response)
    #         return res.response.get('searchObjects')

    #     def alter_maximum_fn(res):
    #         return res.response.get('totalResultCount')

    #     body = {
    #         "count": limit,
    #         "offset": 0,
    #         "hideSearchObjects": True,
    #         "combineResults": False,
    #         "entities": ["CONNECTOR"],
    #         "query": "*"}

    #     obj_list = await datacenter_routes.search_datacenter(
    #         auth=auth,
    #         arr_fn=arr_fn,
    #         alter_maximum_fn=alter_maximum_fn,
    #         body=body,
    #         session=session,
    #         limit=limit,
    #         debug=debug)

    #     if is_close_session:
    #         await session.aclose()

    #     return [DomoConnector._from_str(obj) for obj in obj_list]

    # @classmethod
    # async def update_authorized_domains(cls, auth: DomoAuth,
    #                                     authorized_domain_list: list[str],
    #                                     is_replace_existing_list: bool = False,
    #                                     debug: bool = False):

    #     if not is_replace_existing_list:
    #         existing_domain_list = await cls.get_authorized_domains(auth=auth, debug=debug)

    #         authorized_domain_list.extend(existing_domain_list)

    #     if debug:
    #         print(
    #             f'🌡️ updating authorized domain with {",".join(authorized_domain_list)}')

    #     res = await instance_config_routes.update_authorized_domains(auth=auth,
    #                                                                  authorized_domain_list=authorized_domain_list,
    #                                                                  debug=debug)

    #     if debug:
    #         "update_authorized_domains"
    #         print(res)

    #     if res.status == 200 or res.status == 204:
    #         res = {'authorized_domains': await cls.get_authorized_domains(auth=auth, debug=debug),
    #                'status': 200}

    #     return res

    # @classmethod
    # async def get_applications(cls,
    #                            auth: DomoAuth,
    #                            debug: bool = False, session: httpx.AsyncClient = None, return_raw: bool = False):

    #     res = await application_routes.get_applications(auth=auth,
    #                                                     debug=debug,
    #                                                     session=session)
    #     if debug:
    #         print('Getting Domostats jobs')

    #     if res.status == 200 and not return_raw:
    #         return [DomoApplication._from_json(job) for job in res.response]

    #     if res.status == 200 and return_raw:
    #         return res.response

    # @classmethod
    # async def get_publications(cls,
    #                            auth: DomoAuth,
    #                            debug: bool = False, session: httpx.AsyncClient = None, return_raw: bool = False):

    #     res = await publish_routes.search_publications(auth=auth,
    #                                                    debug=debug,
    #                                                    session=session)
    #     if debug:
    #         print('Getting Publish jobs')

    #     if res.status == 200 and not return_raw:
    #         return await asyncio.gather(*[dmpb.DomoPublication.get_from_id(publication_id=job.get('id'),
    #                                                                        auth=auth) for job in res.response])

    #     if res.status == 200 and return_raw:
    #         return res.response

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
